# Keras LSTM Text Generation

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
text = open('sherlock_homes.txt', 'r').read().lower()
print('text length', len(text))

text length 561852


In [3]:
print(text[:300])

ï»¿adventure i. a scandal in bohemia

i.

to sherlock holmes she is always the woman. i have seldom heard
him mention her under any other name. in his eyes she eclipses
and predominates the whole of her sex. it was not that he felt
any emotion akin to love for irene adler. all emotions, and that
one


## Map chars to integers

In [4]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  59


In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

## Split up into subsequences

In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 187271


In [7]:
print(sentences[:3])
print(next_chars[:3])

['ï»¿adventure i. a scandal in bohemia\n\ni.', 'adventure i. a scandal in bohemia\n\ni.\n\nt', 'enture i. a scandal in bohemia\n\ni.\n\nto s']
['\n', 'o', 'h']


In [8]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [9]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ...  True False False]
  ...
  [ True False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [ True False False ... False False False]
  [ True False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]]
[[ True False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

## Building Model

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [11]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Helper Functions

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [15]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [16]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [17]:
callbacks = [print_callback, checkpoint, reduce_lr]

In [18]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
187271/187271 [==============================] - 225s 1ms/step - loss: 1.9731

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "lge
on the right side of his top-hat to "
lge
on the right side of his top-hat to he wise as the bore with the stor and string that i was a bile that i was a contion with the man with the hadd and the striet with the striet in the stries in the struttle and the striet with the strange with the man with the struttle with the stratter with the striet with the street with the striet which when she with the strunt of the stright of my stright of the string that i shall had been whi
----- diversity: 0.5
----- Generating with seed: "lge
on the right side of his top-hat to "
lge
on the right side of his top-hat to he had putting the stratce, and that is street in the striet man would not the stepe which we had been of the strude
in our in my step withinst in some with the hudied that in had a had become and the corted t

"yes, who was carries of the simple of this son, and then he
----- diversity: 1.0
----- Generating with seed: "ated
visits? was she his client, his fri"
ated
visits? was she his client, his friend. he wild fat, i have faint
on my dest paspen edganed to wateral my him wat
bowed examess. all this one proffesent was as a roneay reasisied to go dad now young time as my sent in  thinkful divinusing st"by colinally would i so did in the firstreatfuldy of
very allting. i ary
b, position was evid take might upon the carrys untal excahed? then he must know your crivance, or now. where we can tha
----- diversity: 1.2
----- Generating with seed: "ated
visits? was she his client, his fri"
ated
visits? was she his client, his frist-, harrarcing hax not fear a mituter."

"pelestodiale i
cromfor of is? otherter at to be toirfowidy
like a bew man of
vullen' head buck"is
was tote kins which i should lro. kand,
'grew iselisle to uror vury, we can to think i awarmed. i out. but in felfeary wor us. did."


In [19]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [20]:
print(generate_text(500, 0.2))

 of something akin to fear had begun
to be a sount of his door and a man in the man of the compants and the commins of the compants of the street. i could he could he married him to be a man which i had a sound of the compant and a street in the compants of the companion, and the country of the little to come and the companion and looked at the street. i have a man which i shall be a man of the comminstance to a some of the man which i could he said to the house of the commins and the man of street in the country and a sound and the c
